In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('/content/crop_yield.csv')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nMissing values:")
print(df.isnull().sum())

# Data Preprocessing
# Handle categorical variables
categorical_cols = ['Region', 'Soil_Type', 'Crop', 'Weather_Condition']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Convert boolean columns to integers
bool_cols = ['Fertilizer_Used', 'Irrigation_Used']
for col in bool_cols:
    df[col] = df[col].astype(int)

# Define features and target
X = df.drop('Yield_tons_per_hectare', axis=1)
y = df['Yield_tons_per_hectare']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale numerical features
numerical_cols = ['Rainfall_mm', 'Temperature_Celsius', 'Days_to_Harvest']
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# Save the model and preprocessing objects
model_data = {
    'model': model,
    'scaler': scaler,
    'label_encoders': label_encoders,
    'feature_names': X.columns.tolist()
}

with open('crop_yield_predictor.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("\nModel saved successfully as 'crop_yield_predictor.pkl'")

# Example of how to use the saved model for prediction
def predict_yield(new_data):
    """
    Function to predict yield for new data
    new_data should be a dictionary with the same structure as the training data
    """
    # Load the saved model
    with open('crop_yield_predictor.pkl', 'rb') as f:
        model_data = pickle.load(f)

    # Create a DataFrame from the input data
    input_df = pd.DataFrame([new_data])

    # Preprocess the input data
    for col in categorical_cols:
        if col in input_df.columns:
            # Handle unseen labels by using the most common category
            try:
                input_df[col] = model_data['label_encoders'][col].transform(input_df[col])
            except:
                input_df[col] = 0  # Default value for unseen categories

    # Convert boolean columns
    for col in bool_cols:
        if col in input_df.columns:
            input_df[col] = input_df[col].astype(int)

    # Scale numerical features
    input_df[numerical_cols] = model_data['scaler'].transform(input_df[numerical_cols])

    # Ensure correct column order
    input_df = input_df[model_data['feature_names']]

    # Make prediction
    prediction = model_data['model'].predict(input_df)
    return prediction[0]

# Test the prediction function with sample data
sample_data = {
    'Region': 'North',
    'Soil_Type': 'Loam',
    'Crop': 'Wheat',
    'Rainfall_mm': 500.0,
    'Temperature_Celsius': 25.0,
    'Fertilizer_Used': True,
    'Irrigation_Used': True,
    'Weather_Condition': 'Sunny',
    'Days_to_Harvest': 120
}

predicted_yield = predict_yield(sample_data)
print(f"\nPredicted yield for sample data: {predicted_yield:.4f} tons per hectare")

Dataset Shape: (1000000, 10)

First 5 rows:
  Region Soil_Type     Crop  Rainfall_mm  Temperature_Celsius  \
0   West     Sandy   Cotton   897.077239            27.676966   
1  South      Clay     Rice   992.673282            18.026142   
2  North      Loam   Barley   147.998025            29.794042   
3  North     Sandy  Soybean   986.866331            16.644190   
4  South      Silt    Wheat   730.379174            31.620687   

   Fertilizer_Used  Irrigation_Used Weather_Condition  Days_to_Harvest  \
0            False             True            Cloudy              122   
1             True             True             Rainy              140   
2            False            False             Sunny              106   
3            False             True             Rainy              146   
4             True             True            Cloudy              110   

   Yield_tons_per_hectare  
0                6.555816  
1                8.527341  
2                1.127443  
3       

In [2]:
import pickle
import pandas as pd

def load_model_and_predict(input_data):
    """
    Load the saved model and make predictions on new data

    Parameters:
    input_data (dict): Dictionary containing the input features

    Returns:
    float: Predicted yield in tons per hectare
    """
    # Load the saved model
    with open('crop_yield_predictor.pkl', 'rb') as f:
        model_data = pickle.load(f)

    # Extract components
    model = model_data['model']
    scaler = model_data['scaler']
    label_encoders = model_data['label_encoders']
    feature_names = model_data['feature_names']

    # Create DataFrame from input data
    input_df = pd.DataFrame([input_data])

    # Preprocess categorical variables
    categorical_cols = ['Region', 'Soil_Type', 'Crop', 'Weather_Condition']
    for col in categorical_cols:
        if col in input_df.columns:
            try:
                # Transform using the saved label encoder
                input_df[col] = label_encoders[col].transform(input_df[col])
            except ValueError:
                # Handle unseen labels by using the most common category (0)
                input_df[col] = 0

    # Convert boolean columns to integers
    bool_cols = ['Fertilizer_Used', 'Irrigation_Used']
    for col in bool_cols:
        if col in input_df.columns:
            input_df[col] = input_df[col].astype(int)

    # Scale numerical features
    numerical_cols = ['Rainfall_mm', 'Temperature_Celsius', 'Days_to_Harvest']
    input_df[numerical_cols] = scaler.transform(input_df[numerical_cols])

    # Ensure correct column order
    input_df = input_df[feature_names]

    # Make prediction
    prediction = model.predict(input_df)
    return prediction[0]

# Example usage
if __name__ == "__main__":
    # Sample input data (replace with your actual data)
    sample_input = {
        'Region': 'North',
        'Soil_Type': 'Loam',
        'Crop': 'Wheat',
        'Rainfall_mm': 500.0,
        'Temperature_Celsius': 25.0,
        'Fertilizer_Used': True,
        'Irrigation_Used': True,
        'Weather_Condition': 'Sunny',
        'Days_to_Harvest': 120
    }

    # Make prediction
    predicted_yield = load_model_and_predict(sample_input)
    print(f"Predicted Yield: {predicted_yield:.4f} tons per hectare")

    # Multiple predictions example
    multiple_inputs = [
        {
            'Region': 'South',
            'Soil_Type': 'Clay',
            'Crop': 'Rice',
            'Rainfall_mm': 800.0,
            'Temperature_Celsius': 28.0,
            'Fertilizer_Used': True,
            'Irrigation_Used': True,
            'Weather_Condition': 'Rainy',
            'Days_to_Harvest': 130
        },
        {
            'Region': 'West',
            'Soil_Type': 'Sandy',
            'Crop': 'Cotton',
            'Rainfall_mm': 300.0,
            'Temperature_Celsius': 30.0,
            'Fertilizer_Used': False,
            'Irrigation_Used': False,
            'Weather_Condition': 'Sunny',
            'Days_to_Harvest': 100
        }
    ]

    print("\nMultiple Predictions:")
    for i, input_data in enumerate(multiple_inputs, 1):
        yield_pred = load_model_and_predict(input_data)
        print(f"Prediction {i}: {yield_pred:.4f} tons per hectare")

Predicted Yield: 5.7322 tons per hectare

Multiple Predictions:
Prediction 1: 7.2133 tons per hectare
Prediction 2: 2.1709 tons per hectare
